# Group Pipeline

## Data Loading

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np

In [ ]:
# Import the dataset
data=pd.read_csv("https://raw.githubusercontent.com/Dilum-Alahakoon/AIML-Project/refs/heads/main/data/raw/post_pandemic_remote_work_health_impact_2025.csv")

# Converting to a dataframe
data_df=pd.DataFrame(data)



In [ ]:
# Display the first five rows
data_df.head()

In [ ]:
# Display the last five rows
data_df.tail()

In [ ]:
# Shape of the dataset

print(f"Shape of the dataset: {data_df.shape}")
print(f"Number of rows : {data_df.shape[0]}")
print(f"Number of rows : {data_df.shape[1]}")

In [ ]:
# Dataset Information
data_df.info()

In [ ]:
# Descriptive statistics of numerical data in dataset
data_df.describe()

In [ ]:
data_df['Mental_Health_Status'].value_counts()

In [ ]:
import matplotlib.pyplot as plt

ax = data_df.hist(
    bins=50,
    figsize=(20, 10),
    color='skyblue',
    edgecolor='black',
    grid=False
)


for a in ax.ravel():
    a.set_facecolor("#f9f9f9")
    a.grid(True, linestyle='--', alpha=0.5)
    a.tick_params(axis='x', labelsize=10)
    a.tick_params(axis='y', labelsize=10)
    a.set_title(a.get_title(), fontsize=12, fontweight='bold')


plt.suptitle("Feature Distributions", fontsize=20, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
categorical_features=data_df.select_dtypes(include=['object']).columns.tolist()
numerical_features=data_df.select_dtypes(include=[np.number]).columns.tolist()
print(f"Categorical Features: {categorical_features}\n")
print(f"Numerical Features: {numerical_features}")

In [ ]:
for col in categorical_features:
  print(data_df[col].value_counts())
  print()

In [ ]:
import seaborn as sns

for col in categorical_features:
  plt.figure(figsize=(8,4))
  sns.countplot(y=col,data=data_df,order=data_df[col].value_counts().index,palette='viridis')
  plt.title(f"Distribution of {col}")
  plt.tight_layout()
  plt.show()

## Handling Missing Data

In [ ]:
# Identifying the missing data

data_df.isnull().sum()

In [ ]:
# Heatmap for missing values

import seaborn as sns
plt.figure(figsize=(12,6))
sns.heatmap(data_df.isnull(),cbar=False)
plt.title("Missing Values Heatmap")
plt.show()

missing_counts=data_df.isnull().sum().sort_values(ascending=False)
print(missing_counts[missing_counts>0])

In [ ]:
# Bar graph for the missing values

missing_counts=data_df.isnull().sum()
missing_cols=missing_counts[missing_counts>0].sort_values(ascending=False)

if len(missing_cols)==0:
  print("No missing values in the dataset!")
else:
  plt.figure(figsize=(10,6))
  sns.barplot(x=missing_cols.index, y=missing_cols.values,palette='viridis')
  plt.xticks(rotation=45,ha='right')
  plt.ylabel('Number of Missing Values')
  plt.title('Missing Values Count by Feature')
  plt.tight_layout()
  plt.show()

In [ ]:
# Drop the missing values of the target variable(Mental Helth Status)

data_df.dropna(subset=['Mental_Health_Status'],inplace=True)

In [ ]:
data_df.isnull().sum()

In [ ]:
# data shape after removing the missing values of the target variable

print(f"\nNumber of rows & columns after removing the missing values of the target variable: {data_df.shape}")

In [ ]:
# hetamap after removing the missing values of the target variable

import seaborn as sns
plt.figure(figsize=(12,6))
sns.heatmap(data_df.isnull(),cbar=False)
plt.title('Missing Values Heatmap')
plt.show()


missing_counts=data_df.isnull().sum().sort_values(ascending=False)
print(missing_counts[missing_counts>0])

In [ ]:
# Bar graph after removing the target variable (Mental Health Status) missing values

missing_counts=data_df.isnull().sum()
missing_cols=missing_counts[missing_counts>0].sort_values(ascending=False)

if len(missing_cols)==0:
  print("No missing values in the dataset!")
else:
  plt.figure(figsize=(10,6))
  sns.barplot(x=missing_cols.index, y=missing_cols.values,palette='viridis')
  plt.xticks(rotation=45,ha='right')
  plt.ylabel('Number of Missing Values')
  plt.title('Missing Values Count by Feature')
  plt.tight_layout()
  plt.show()

After removing the missing values of the target variable there are 203 missing values in the Phisical Health Issues because of that we need to handle that missing values

In [ ]:
# getting the count of the physical health issues count

data_df['Physical_Health_Issues'].value_counts()

In [ ]:
X=data_df.drop(labels=['Physical_Health_Issues'],axis=1)
y=data_df['Mental_Health_Status']

In [ ]:
numerical_features=X.select_dtypes(include=[np.number]).columns.tolist()

In [ ]:
# creating a new dataframe only using numerical features
test_dataframe=pd.DataFrame(X[numerical_features])

In [ ]:
# to this dataframe attaching the target variable as Physical_Health_Issues
test_dataframe['target']=data_df['Physical_Health_Issues']

In [ ]:
test_dataframe.head()

In [ ]:
# calculating the missing values in the new dataframe
# here in the data frame in the target variable there are 208 missing values

test_dataframe.isnull().sum()

In [ ]:
# here i split the newly formed dataset into two parts known part contain the rows with not missing values in the target variable and missing part it contains rows with missing values in the target variable

known=test_dataframe[test_dataframe['target'].notna()]
missing=test_dataframe[test_dataframe['target'].isna()]

In [ ]:
known.head()

In [ ]:
# here in the known column we are split into input and and predicting variable

mis_X=known.drop('target',axis=1)
mis_y=known['target']

In [ ]:
# using randomforest classifier to train a model using abobe splitted data

from sklearn.ensemble import RandomForestClassifier

model=RandomForestClassifier(random_state=42)
model.fit(mis_X,mis_y)

In [ ]:
# seperating the unknown part and drop the target variable
X_unknown=missing.drop('target',axis=1)

# predicting the values for X_unknown
predicted=model.predict(X_unknown)

In [ ]:
# attaching the predicted part to the dataframe

test_dataframe.loc[test_dataframe['target'].isna(),'target']=predicted

In [ ]:
data_df.drop('Physical_Health_Issues',axis=1,inplace=True)

In [ ]:
data_df['Physical_Health_Issues']=test_dataframe['target']

In [ ]:
data_df['Physical_Health_Issues'].value_counts()

In [ ]:
# After removing the missing values

data_df.isnull().sum()



After predict the missing values from the numerical features there are no null values in the physical health issues feature and there are no null values in other fields also